In [31]:
from Crypto.Util.number import bytes_to_long, long_to_bytes, getPrime
import random

class Challenge():
    def __init__(self):
        self.before_input = "Come back as much as you want! You'll never get my flag.\n"
        self.p = getPrime(20)
        self.q = getPrime(20)
        assert self.p != self.q
        self.N = self.p * self.q
        self.e = 11

    def pad(self, flag):
        m = bytes_to_long(flag)
        a = random.randint(2, self.N)
        b = random.randint(2, self.N)
        return (a, b), a*m+b

    def encrypt(self, flag):
        pad_var, pad_msg = self.pad(flag)
        encrypted = (pow(pad_msg, self.e, self.N), self.N) # (a*m+b)^11 mod N, N
        return pad_var, encrypted

    def challenge(self, your_input):
        if not 'option' in your_input:
            return {"error": "You must send an option to this server"}

        elif your_input['option'] == 'get_flag':
            pad_var, encrypted = self.encrypt(FLAG)
            return {"encrypted_flag": encrypted[0], "modulus": encrypted[1], "padding": pad_var}

        else:
            return {"error": "Invalid option"}

In [29]:
a, b, C, Ns, fs = [], [], [], [], []
MSG = b'\xccaa'
for i in range(12):
    c = Challenge()
    var, encrypted = c.encrypt(MSG)
    _a_, _b_ = var
    _C_, _N_ = encrypted
    
    _f_ = modPoly(powPoly([_b_, _a_], 11), _N_)
    _f_[0] -= _C_
    _f_[0] %= _N_
    _f_[0]  = int(_f_[0])
    
    _g_ = makeMonic(_f_, _N_)
    _v_ = getValue(_g_, 3)
    fs.append(_g_)
    
    a.append(_a_)
    b.append(_b_)
    Ns.append(_N_)
    C.append(_C_)

In [33]:
##################################################### Simple transformation ############################################################# 

def trimZero(f):
    while len(f) > 0 and f[-1] == 0:
        f.pop()
    return f

def fillZero(f, lenFill):
    assert len(f) <= lenFill
    return f + [0] * (lenFill - len(f))

def makeMonic(f, modulus):
    r = f.copy()
    leadCoefficientInverse = inverse_mod(r[-1], modulus)
    for i in range(len(r)):
        r[i] *= leadCoefficientInverse
        r[i] %= modulus
        r[i] = int(r[i])
    return r

##################################################### Arithmetics: +, *, %, ^ #############################################################  

def modPoly(f, modulus):
    r = f.copy()
    for i in range(len(f)):
        r[i] %= modulus
        r[i] = int(r[i])
    return trimZero(r)

def addPoly(f1, f2):
    f3 = [0] * max(len(f1), len(f2))
    for i in range(len(f3)):
        if i < min(len(f1), len(f2)):
            f3[i] = f1[i] + f2[i]
        elif len(f1) < len(f2):
            f3[i] = f2[i]
        else:
            f3[i] = f1[i]
    return trimZero(f3)

def mulPoly(f1, f2):
    f3 = [0] * (len(f1) + len(f2))
    for i2 in range(len(f2)):
        for i1 in range(len(f1)):
            f3[i1 + i2] += f1[i1] * f2[i2]
    return trimZero(f3)

def powPoly(f, p):
    r = [1]
    for i in range(p):
        r = mulPoly(r, f)
    return r

##################################################### Evaluate f(x) at x = X, solve f(x) = 0... ############################################################# 

def getRoots(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(f)):
        fInPolyRep += f[i] * _x_ ^ i
    try:
#         print("[i] Finding roots of {}...".format(fInPolyRep))
        return fInPolyRep.roots()
    except:
        return []
    
def getValue(f, x):
    value = 0
    for coefficient in f[::-1]:
        value += coefficient
        value *= x
    return value // x
    
def genPoly(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(f)):
        fInPolyRep += f[i] * _x_ ^ i
    return fInPolyRep

##################################################### More complicated stuffs... ############################################################# 

def coppersmith(f, modulus, m, X=None):    
    # Assure monic
    assert f[-1] == 1
    
    # Get degree & set X if necessary
    degree = len(f) - 1
    assert degree >= 1
    if not X:
        X = int(modulus ^ (1 / degree))
        if degree == 1:
            X -= 1
#         print('[No upperbound is set so the algorithm set it to {}]'.format(X))
    
#     print("[] Solving {} = 0 mod {} \n\tfor x < {} \n\tfor m = {}".format(genPoly(f), modulus, X, m))
    
    # Get dim
    dimension = (degree) * (m + 1)

    # Constructs a matrix.
    M = []
    for v in range( 0, m+1 ):
        for u in range( 0, degree ):
            # Generate g(x) function from f(x)
            g = mulPoly( mulPoly( [modulus^(m-v)], [0]*u+[1] ), powPoly(f, v) )
            
            # Create g(xX) from g(x)
            for i in range(len(g)):
                g[i] *= X^i
            
            # Append to row
            M.append( fillZero(g, dimension) )
#     print("[i] Generated a matrix size {}x{}".format(len(M), len(M[0])))
    
    # Convert to matrix
    M = matrix(M)
#     print("[] M:\n")
#     print(M)
    
    # Apply LLL
    M = M.LLL()
#     print("\n[] M.LLL():\n")
#     print(M); print('\n')
    
    
    # Scroll through the shortest vectors as the new polynomials
    roots = []
    for row in M:
        f = trimZero([int(col) for col in row])
        for i in range(len(f)):
            assert f[i] % (X^i) == 0
            f[i] //= X^i
#         print("[] f: {}, row: {}".format(f, row))
        roots += getRoots(f)
    
    # Return unique X component of roots
    return list(dict.fromkeys([root[0] for root in roots]))

def hastad(fs, Ns, m, X=None):
    for i in range(len(Ns)):
        for j in range(i+1, len(Ns)):
            if gcd(Ns[i], Ns[j]) != 1:
                raise ValueError("The Ns must be all coprime!")
    
    # Setup variables
    Ts    = [1] * len(fs)
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
        
    # Generate Ti, Chinese Remainder Coefficients
    for i in range(len(fs)):
        Ts[i] *= ProdN // Ns[i]
        Ts[i] *= int(inverse_mod(int(Ts[i] % Ns[i]), Ns[i]))
        
    # Make sure T serves our criteria
    for i in range(len(Ts)):
        for j in range(len(Ns)):
            if (i == j and Ts[i] % Ns[j] != 1) or (i != j and Ts[i] % Ns[j] != 0):
                assert False
        
    # Generate f(x) from fi(x)s
    f = [0]
    for i in range(len(fs)):
        f = addPoly(f, mulPoly([Ts[i]], fs[i]))
    f = modPoly(f, ProdN)
    
    # Make sure we actually had monic polynomial
    assert f[-1] == 1
    
    # Using coppersmith to solve
    return coppersmith(f, ProdN, m, X)

def hastad2(fs, Ns, beta=1.0, eps=1/8):
    for i in range(len(Ns)):
        for j in range(i+1, len(Ns)):
            if gcd(Ns[i], Ns[j]) != 1:
                raise ValueError("The Ns must be all coprime!")
    
    # Setup variables
    Ts    = [1] * len(fs)
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
        
    # Generate Ti, Chinese Remainder Coefficients
    for i in range(len(fs)):
        Ts[i] *= ProdN // Ns[i]
        Ts[i] *= int(inverse_mod(int(Ts[i] % Ns[i]), Ns[i]))
        
    # Make sure T serves our criteria
    for i in range(len(Ts)):
        for j in range(len(Ns)):
            if (i == j and Ts[i] % Ns[j] != 1) or (i != j and Ts[i] % Ns[j] != 0):
                assert False
        
    # Generate f(x) from fi(x)s
    f = [0]
    for i in range(len(fs)):
        f = addPoly(f, mulPoly([Ts[i]], fs[i]))
    f = modPoly(f, ProdN)
    
    # Make sure we actually had monic polynomial
    assert f[-1] == 1
    
    # Using coppersmith to solve
    P = genPoly(f).change_ring(Zmod(ProdN))
    return P.small_roots(beta=beta, eps=eps)

In [44]:
def finding(m=0x10000, l=0x1000, r=0x10000):
    a, b, C, Ns, fs = [], [], [], [], []
    MSG = long_to_bytes(m)
    for i in range(12):
        c = Challenge()
        var, encrypted = c.encrypt(MSG)
        _a_, _b_ = var
        _C_, _N_ = encrypted

        _f_ = modPoly(powPoly([_b_, _a_], 11), _N_)
        _f_[0] -= _C_
        _f_[0] %= _N_
        _f_[0]  = int(_f_[0])

        _g_ = makeMonic(_f_, _N_)
        _v_ = getValue(_g_, 3)
        fs.append(_g_)

        a.append(_a_)
        b.append(_b_)
        Ns.append(_N_)
        C.append(_C_)
    
    while (l != r and l != r-1) and l < r:
#         print(l, r)
        roots = hastad(fs, Ns, 3, (l+r) // 2)
        if not (len(roots) > 0 and m in roots):
            l = (l+r) // 2
        else:
            r = (l+r) // 2
    return l, r

finding()

(4096, 4097)

In [ ]:
def finding2(m=0x10000):
    a, b, C, Ns, fs = [], [], [], [], []
    MSG = long_to_bytes(m)
    for i in range(12):
        c = Challenge()
        var, encrypted = c.encrypt(MSG)
        _a_, _b_ = var
        _C_, _N_ = encrypted

        _f_ = modPoly(powPoly([_b_, _a_], 11), _N_)
        _f_[0] -= _C_
        _f_[0] %= _N_
        _f_[0]  = int(_f_[0])

        _g_ = makeMonic(_f_, _N_)
        _v_ = getValue(_g_, 3)
        fs.append(_g_)

        a.append(_a_)
        b.append(_b_)
        Ns.append(_N_)
        C.append(_C_)
    
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
    return hastad2(fs, Ns, min(Ns) / ProdN, min(Ns) / ProdN / 8)

finding2()